In [1]:
### Getting the total number of confirmed cases in the top 20 affected countries and all the states in the US.
### purpose: to run the cron-job every hour using HitchHiqe's resources to perform ridge based lambda regression 
### for further classification and improving the accuracy of the algorithm in better prediciton for the app.


from selenium import webdriver
import pandas as pd
import os
import re
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import random
from datetime import datetime
import collections

In [2]:
URL = "https://www.arcgis.com/apps/opsdashboard/index.html#/85320e2ea5424dfaaa75ae62e5c06e61"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless = True
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
br = webdriver.Chrome(executable_path=r"C:\Users/qasim/Downloads/chromedriver_win32_v80/chromedriver.exe",options=chrome_options)
br.get(URL)
time.sleep(3)

In [4]:
data = br.find_element_by_id("ember21").text.split("\n")[:6]

In [5]:
def extract_num(string):
    string = string.split(" ")[0]
    return int(string.replace(",", ""))

In [6]:
logistics = {"CONFIRMED" : extract_num(data[1]), "DEATHS" : extract_num(data[3]), "RECOVERED" : extract_num(data[5])}

In [7]:
logistics

{'CONFIRMED': 214894, 'DEATHS': 8732, 'RECOVERED': 83313}

In [8]:
data = br.find_element_by_id("ember107").click()

In [9]:
TIME = str(datetime.now())
TIME = ':'.join(TIME.split(":")[:-1])

In [10]:
root = br.find_element_by_id("ember60")

In [11]:
br.find_element_by_id("ember66").screenshot("Images/"+ TIME.split(" ")[0] +".png")

True

In [12]:
data = root.text.split("\n")[1:100]

In [13]:
case = {}

In [14]:
for x in data:
    x = x.split(" ")
    country = " ".join(x[1:]).replace("*", "").strip()
    cases = int(x[0].replace(",",""))
    case[country] = cases

In [15]:
case = collections.OrderedDict(sorted(case.items()))

In [16]:
raw_data = {**logistics, **case}

In [17]:
df = pd.DataFrame.from_dict(raw_data, orient="index", columns=[TIME]).reset_index()

In [18]:
df = df.rename(columns={"index":"Country"})

In [19]:
df.head()

,Country,2020-03-18 17:44
0,CONFIRMED,214894
1,DEATHS,8732
2,RECOVERED,83313
3,Afghanistan,22
4,Albania,59


In [20]:
df.to_csv("Confirmed_case.csv", index=False)